# Алгоритм иерархической кластеризации с алгоритмом объединения "single-link clustering" (SLINK)

Алгоритм начинает работу, считая каждую точку отдельным кластером. На каждой итерации он находит два максимально близких кластера (с минимальным расстоянием между ближайшими элементами) и сливает их в один. Это продолжается, пока все точки не окажутся в одном кластере.

Сначала алгоритм рассчитывает матрицу расстояний $ D $ (для удобства диагональ заполняется бесконечностями). Также для каждой точки в двух дополнительных массивах хранится индекс ближайшего соседа и расстояние до него.

Это занимает $O(n^2)$ времени.

На каждой итерации: 
 - находится минимум в массиве расстояний до ближайшего соседа и две соответствующие точки $i$ и $j$
 - в массиве расстояний до ближайшего соседа на j-том месте ставится бесконечность, что соответствует удалению $j$ из рассмотрения
 - пересчитывается i-тая строчка матрицы расстояний: $D(i,x) = min( D(i,x), D(j,x) )$. Теперь i-тая строчка соответствует кластеру $ij$
 - для кластера $ij$ находится новый ближайший сосед и расстояние до него и записывается в i-тую ячейку соответствующих массивов
 - если для какой-то точки ближайшим соседом была $j$, то для нее ближайшим соседом становится $i$

$O(n)$ итераций, на каждой из них тратится $O(n)$ времени на поиск минимума в массивах и обновление строки в матрице.<br>
Итого сложность по времени $O(n^2)$

In [1]:
import numpy as np

def distance(a,b):
    return abs(a - b)

def proximity_matrix(x, inf):  
    proximity = np.eye(len(x))
    for i,a in enumerate(x):
        for j,b in enumerate(x):
            if i == j:
                proximity[i,j] = inf
            else:
                proximity[i,j] = distance(a,b)
    return proximity

def SLINKcluster(x, inf):
    clusters = [[i] for i in x]

    proximity = proximity_matrix(x, inf)
    dist_list = np.min(proximity, axis=0)
    nearest_list = np.argmin(proximity, axis=0) 
    
    for step in range(len(x)-1):
        i = np.argmin(dist_list)
        j = nearest_list[i]

        dist_list[j] = inf
        
        for k,a in enumerate(x):
            if k == i or k == j:
                d = inf
            else:
                d = min(proximity[i,k], proximity[j,k])
            proximity[i,k] = d
            proximity[k,j] = inf 
        
        dist_list[i] = min(proximity[i])
        nearest_list[i] = np.argmin(proximity[i])

        for k,n in enumerate(nearest_list):
            if n == j: 
                nearest_list[k] = i

        clusters[i] = [clusters[i], clusters[j]]
    return clusters[i]

Недостаток этого метода в том, что он производит "длинные" кластеры, в которых расстояние между соседями мало, но расстояние между элементами на двух противоположных концах кластера велико.

Также метод имеет проблему "цепных кластеров", в которых к кластеру каждый раз добавляется одиночный элемент.<br>
Пример: на точках $x = [1, 2, 4, 6, 8]$ получатся кластеры $[[[[1,2],4],6],8]$

Алгоритм SLINK является оптимальным для произвольного кластеризируемого множества, варьироваться в нем будет лишь способ измерения расстояния между точками в матрице расстояний.

Но для специфических постановок задачи алгоритм может быть улучшен.

# Алгоритм иерархической кластеризации для точек на прямой
Заметим, что на прямой ближайшими соседями точки могут быть только ее непосредственные соседи, а значит можно хранить расстояние только до соседа справа. К тому же только соседние кластеры могут сливаться.

Алгоритм сортирует массив точек. Заполняется массив расстояний между соседними точками и на его основе строится очередь с приоритетом. Также заполняется массив, который для каждой точки хранит индекс "образующей" кластера - минимального элемента кластера, к которому непосредственно принадлежит эта точка (или кластер, если сама точка является "образующей").<br>
Сложность: $O(n*log~n)$ из-за сортировки.

На каждой итерации:
 - из очереди расстояний выталкивается минимальный элемент и определяются ближайшие точки $i$ и $i+1$
 - определяется индекс $n_i$ образующей максимального кластера, к которому принадлежит $i$ (подъемом к корню по дендрограмме на текущем шаге)
 - сливаются кластеры с образующими $n_i$ и $i+1$ (т.к. $i+1$ всегда образующая своего кластера)
 - для точки $i+1$ обновляется ссылка на образующую кластера
 
Сложность: $O(n)$ итераций, на каждой $O(log~n)$ операций ($~O(n)$ в худшем случае слияния цепного кластера вида $[1,[5,[8,[10,11]]]]$ и $[20...]~$).

Итого сложность $O(n*log~n)$.


In [2]:
import numpy as np
import heapq
def linecluster(x):
    x.sort()
    clusters = [[i] for i in x]
    
    distances = [(abs(x[i] - x[i+1]), i)  for i in range(len(x)-1)]
    heapq.heapify(distances) 
    
    nearest = [i for i in range(len(x))]

    for step in range(len(x)-1):
        t = heapq.heappop(distances) 
        i = t[-1]    

        ni = i
        while nearest[ni] < ni:
            ni = nearest[ni]
        clusters[ni] = [clusters[ni], clusters[i+1]]
        
        nearest[i+1] = ni
                
    return clusters[ni]

In [3]:
import random
x = [random.randrange(100) for i in range(10)]
# x = [float(a) for a in input().split()]

print(linecluster(x))

[[[[5], [10]], [[23], [[[31], [35]], [40]]]], [[[70], [70]], [[79], [82]]]]
